In [1]:
import os
import pcse
import yaml
import h5py
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
from pcse.models import Wofost71_PP, Wofost71_WLP_FD
from pcse.base import ParameterProvider
from pcse.db import NASAPowerWeatherDataProvider
from pcse import fileinput as fi
from pcse.util import WOFOST71SiteDataProvider

In [2]:
agro_yaml = """
Version: 1.0
AgroManagement:
- {sim_start_date}:
    CropCalendar:
        crop_name: maize
        variety_name: Maize_VanHeemst_1988
        crop_start_type: sowing
        crop_start_date: {crop_start_date}
        crop_end_type: earliest
        crop_end_date: {crop_end_date}
        max_duration: 300
    TimedEvents: 
    -   event_signal: apply_n
        name:  Nitrogen application table
        comment: All nitrogen amounts in g N m-2
        events_table:
        - {fert_date}: {{amount: {n_rate}, recovery: 0.7}}
    StateEvents: null
"""

In [3]:
pts = gpd.read_file('../../CIAT/data/BEM_2019_Chiapas_Final_Clean.gpkg')
pts['site'] = ['CH{0:05d}'.format(i + 1) for i in range(len(pts))]
pts['lon'] = pts.centroid.x
pts['lat'] = pts.centroid.y
pts['ref_date'] = pd.to_datetime(pts.fecha_de_siembra)
pts['nrate'] = pts.nitrogen/10


In [4]:
soilpts = gpd.read_file('../data/CH_Soils.gpkg')
soilpts.geometry = soilpts.to_crs('epsg:32715').buffer(5000).to_crs(pts.crs)
pts = gpd.sjoin(pts, soilpts, how = 'left')

/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
for i in trange(len(pts)):
    if np.isnan(pts.index_right.values[i]):
        continue
    soilfile = '../data/soils/{}.sol'.format(pts.SoilProfil.values[i])
    soildata = fi.CABOFileReader(soilfile)
    soildata.update({'RDMSOL': 200, 'SM0': 0.4, 'SMW': 0.1})

    cropdata = fi.YAMLCropDataProvider()
    cropdata.set_active_crop('maize', 'Maize_VanHeemst_1988')

    sitedata = WOFOST71SiteDataProvider(WAV=100, CO2=360)

    parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)


    wfile = '/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{}_npf.csv'.format(pts.site.values[i])
    wdp = fi.CSVWeatherDataProvider(wfile)

    # get new start date
    crop_start_date = pts.ref_date.dt.date.values[i]
    sim_start_date = crop_start_date - dt.timedelta(days = 90)
    crop_end_date = crop_start_date + dt.timedelta(days = 210)
    fert_date = crop_start_date + dt.timedelta(days = 30)
    n_rate = 1

    # update agromanagement with new start date and load it with yaml.load
    tmp = agro_yaml.format(sim_start_date = sim_start_date,
                           crop_start_date = crop_start_date,
                           crop_end_date = crop_end_date,
                           fert_date = fert_date,
                           n_rate = n_rate)
    agromanagement = yaml.load(tmp, yaml.FullLoader)

    try:
        wofsim = Wofost71_WLP_FD(parameters, wdp, agromanagement)
        wofsim.run_till_terminate()
        print('Finished {}'.format(pts.site.values[i]))
        output = wofsim.get_output()
        df = pd.DataFrame(output).set_index("day")
        df.to_hdf('../data/PSCE.h5', key = pts.site.values[i])
        summary_output = wofsim.get_summary_output()
        msg = "Reached maturity at {DOM} with total biomass {TAGP:0.2f} kg/ha and a yield of {TWSO:0.2f} kg/ha."
        print(msg.format(**summary_output[0]))
    except pcse.exceptions.WeatherDataProviderError as e:
        print('Problem with {}'.format(pts.site.values[i]))
        print(e)
        print(crop_start_date)



Finished CH00252
Reached maturity at 2012-08-20 with total biomass 19872.21 kg/ha and a yield of 9847.69 kg/ha.
Finished CH00253
Reached maturity at 2012-09-08 with total biomass 19917.08 kg/ha and a yield of 9335.14 kg/ha.
Finished CH00254
Reached maturity at 2012-09-15 with total biomass 19972.85 kg/ha and a yield of 9276.22 kg/ha.
Finished CH00255
Reached maturity at 2012-09-10 with total biomass 19818.96 kg/ha and a yield of 9294.89 kg/ha.
Finished CH00256
Reached maturity at 2012-09-11 with total biomass 19800.42 kg/ha and a yield of 9280.98 kg/ha.
Finished CH00257
Reached maturity at 2012-10-03 with total biomass 20395.95 kg/ha and a yield of 9476.11 kg/ha.
Finished CH00258
Reached maturity at 2012-10-18 with total biomass 19742.98 kg/ha and a yield of 8934.07 kg/ha.
Finished CH00259
Reached maturity at 2012-11-03 with total biomass 24735.99 kg/ha and a yield of 10599.67 kg/ha.
Finished CH00260
Reached maturity at 2012-09-10 with total biomass 19812.02 kg/ha and a yield of 9294.3

Finished CH00323
Reached maturity at 2012-10-16 with total biomass 24498.82 kg/ha and a yield of 10948.10 kg/ha.
Finished CH00324
Reached maturity at 2012-10-16 with total biomass 24498.82 kg/ha and a yield of 10948.10 kg/ha.
Finished CH00325
Reached maturity at 2012-10-08 with total biomass 25215.83 kg/ha and a yield of 11290.23 kg/ha.
Finished CH00325
Reached maturity at 2012-10-08 with total biomass 25215.83 kg/ha and a yield of 11290.23 kg/ha.
Finished CH00326
Reached maturity at 2012-09-05 with total biomass 23910.24 kg/ha and a yield of 12230.64 kg/ha.
Finished CH00327
Reached maturity at 2012-11-03 with total biomass 22533.26 kg/ha and a yield of 9793.10 kg/ha.
Finished CH00328
Reached maturity at 2012-11-17 with total biomass 23894.85 kg/ha and a yield of 10328.59 kg/ha.
Finished CH00329
Reached maturity at 2012-11-09 with total biomass 24196.01 kg/ha and a yield of 10158.28 kg/ha.
Finished CH00330
Reached maturity at 2012-09-23 with total biomass 25549.89 kg/ha and a yield of 

Finished CH00402
Reached maturity at 2012-11-09 with total biomass 26687.57 kg/ha and a yield of 11511.92 kg/ha.
Finished CH00403
Reached maturity at 2012-09-27 with total biomass 26282.38 kg/ha and a yield of 11877.10 kg/ha.
Finished CH00404
Reached maturity at 2012-09-25 with total biomass 26304.51 kg/ha and a yield of 11883.41 kg/ha.
Finished CH00405
Reached maturity at 2012-10-20 with total biomass 21713.96 kg/ha and a yield of 9445.51 kg/ha.
Finished CH00406
Reached maturity at 2012-11-07 with total biomass 21303.92 kg/ha and a yield of 8871.53 kg/ha.
Finished CH00407
Reached maturity at 2012-10-01 with total biomass 26430.28 kg/ha and a yield of 11845.69 kg/ha.
Finished CH00408
Reached maturity at 2012-09-20 with total biomass 26318.98 kg/ha and a yield of 11903.39 kg/ha.
Finished CH00409
Reached maturity at 2012-10-17 with total biomass 22084.67 kg/ha and a yield of 9513.79 kg/ha.
Finished CH00410
Reached maturity at 2012-09-25 with total biomass 22613.53 kg/ha and a yield of 10

Finished CH00480
Reached maturity at 2012-09-22 with total biomass 22644.34 kg/ha and a yield of 10466.28 kg/ha.
Finished CH00481
Reached maturity at 2012-10-05 with total biomass 21965.35 kg/ha and a yield of 10046.57 kg/ha.
Finished CH00482
Reached maturity at 2012-10-15 with total biomass 23342.88 kg/ha and a yield of 10432.07 kg/ha.
Finished CH00483
Reached maturity at 2012-09-29 with total biomass 22862.13 kg/ha and a yield of 10478.27 kg/ha.
Finished CH00484
Reached maturity at 2012-10-22 with total biomass 22532.16 kg/ha and a yield of 9951.11 kg/ha.
Finished CH00485
Reached maturity at 2012-10-22 with total biomass 22532.16 kg/ha and a yield of 9951.11 kg/ha.
Finished CH00486
Reached maturity at 2012-10-10 with total biomass 23458.74 kg/ha and a yield of 10596.69 kg/ha.
Finished CH00487
Reached maturity at 2012-10-08 with total biomass 21929.55 kg/ha and a yield of 10013.46 kg/ha.
Finished CH00488
Reached maturity at 2012-10-15 with total biomass 23342.88 kg/ha and a yield of 1

Finished CH00558
Reached maturity at 2012-10-10 with total biomass 21824.41 kg/ha and a yield of 9959.72 kg/ha.
Finished CH00559
Reached maturity at 2012-10-03 with total biomass 21916.61 kg/ha and a yield of 10083.83 kg/ha.
Finished CH00560
Reached maturity at 2012-12-02 with total biomass 20716.38 kg/ha and a yield of 7095.67 kg/ha.
Finished CH00562
Reached maturity at 2012-08-27 with total biomass 21271.76 kg/ha and a yield of 10534.96 kg/ha.
Finished CH00563
Reached maturity at 2012-08-27 with total biomass 21271.76 kg/ha and a yield of 10534.96 kg/ha.
Finished CH00564
Reached maturity at 2012-11-29 with total biomass 21191.20 kg/ha and a yield of 7496.74 kg/ha.
Finished CH00565
Reached maturity at 2012-10-04 with total biomass 20340.73 kg/ha and a yield of 9483.73 kg/ha.
Finished CH00566
Reached maturity at 2012-10-15 with total biomass 23342.83 kg/ha and a yield of 10439.56 kg/ha.
Finished CH00567
Reached maturity at 2012-08-10 with total biomass 15590.84 kg/ha and a yield of 101

...

Finished CH03965
Reached maturity at 2018-09-19 with total biomass 18452.37 kg/ha and a yield of 11294.92 kg/ha.
Finished CH03966
Reached maturity at 2018-09-16 with total biomass 18513.35 kg/ha and a yield of 11212.68 kg/ha.
Finished CH03967
Reached maturity at 2018-09-01 with total biomass 16138.21 kg/ha and a yield of 8143.35 kg/ha.
Finished CH03968
Reached maturity at 2018-11-27 with total biomass 21024.61 kg/ha and a yield of 8869.07 kg/ha.
Finished CH03969
Reached maturity at 2018-09-22 with total biomass 18213.34 kg/ha and a yield of 11283.05 kg/ha.
Finished CH03969
Reached maturity at 2018-09-22 with total biomass 18213.34 kg/ha and a yield of 11283.05 kg/ha.
Finished CH03970
Reached maturity at 2018-09-28 with total biomass 18000.13 kg/ha and a yield of 10844.55 kg/ha.
Finished CH03970
Reached maturity at 2018-09-28 with total biomass 18000.13 kg/ha and a yield of 10844.55 kg/ha.
Finished CH03972
Reached maturity at 2018-09-14 with total biomass 18675.63 kg/ha and a yield of 1

Finished CH04030
Reached maturity at 2018-10-13 with total biomass 20396.02 kg/ha and a yield of 11015.23 kg/ha.
Finished CH04031
Reached maturity at 2018-10-09 with total biomass 19968.60 kg/ha and a yield of 11091.51 kg/ha.
Finished CH04032
Reached maturity at 2018-10-24 with total biomass 18744.87 kg/ha and a yield of 10189.98 kg/ha.
Finished CH04033
Reached maturity at 2018-10-19 with total biomass 19569.45 kg/ha and a yield of 10249.85 kg/ha.
Finished CH04034
Reached maturity at 2018-09-14 with total biomass 19465.72 kg/ha and a yield of 10759.29 kg/ha.
Finished CH04035
Reached maturity at 2018-09-28 with total biomass 19762.97 kg/ha and a yield of 10802.81 kg/ha.
Finished CH04036
Reached maturity at 2018-09-16 with total biomass 19532.34 kg/ha and a yield of 10936.42 kg/ha.
Finished CH04037
Reached maturity at 2018-09-16 with total biomass 19532.34 kg/ha and a yield of 10936.42 kg/ha.
Finished CH04038
Reached maturity at 2018-08-03 with total biomass 16898.04 kg/ha and a yield of

Finished CH04099
Reached maturity at 2018-10-04 with total biomass 19808.01 kg/ha and a yield of 10619.20 kg/ha.
Finished CH04100
Reached maturity at 2018-09-16 with total biomass 19540.78 kg/ha and a yield of 10938.30 kg/ha.
Finished CH04101
Reached maturity at 2018-09-12 with total biomass 18092.43 kg/ha and a yield of 10459.15 kg/ha.
Finished CH04102
Reached maturity at 2018-10-12 with total biomass 11529.74 kg/ha and a yield of 7460.45 kg/ha.
Finished CH04103
Reached maturity at 2018-09-25 with total biomass 19902.27 kg/ha and a yield of 10940.54 kg/ha.
Finished CH04104
Reached maturity at 2018-09-25 with total biomass 19902.27 kg/ha and a yield of 10940.54 kg/ha.
Finished CH04105
Reached maturity at 2018-09-07 with total biomass 21128.04 kg/ha and a yield of 8754.16 kg/ha.
Finished CH04106
Reached maturity at 2018-09-17 with total biomass 21546.82 kg/ha and a yield of 10922.00 kg/ha.
Finished CH04107
Reached maturity at 2018-09-20 with total biomass 20256.39 kg/ha and a yield of 1

Finished CH04171
Reached maturity at 2018-09-22 with total biomass 19888.49 kg/ha and a yield of 11067.23 kg/ha.
Finished CH04172
Reached maturity at 2018-09-22 with total biomass 19888.49 kg/ha and a yield of 11067.23 kg/ha.
Finished CH04172
Reached maturity at 2018-09-22 with total biomass 19888.49 kg/ha and a yield of 11067.23 kg/ha.
Finished CH04173
Reached maturity at 2018-10-18 with total biomass 18528.99 kg/ha and a yield of 11318.30 kg/ha.
Finished CH04174
Reached maturity at 2018-10-21 with total biomass 18063.49 kg/ha and a yield of 11154.32 kg/ha.
Finished CH04175
Reached maturity at 2018-10-22 with total biomass 18035.30 kg/ha and a yield of 11151.14 kg/ha.
Finished CH04176
Reached maturity at 2018-10-17 with total biomass 18861.69 kg/ha and a yield of 11431.16 kg/ha.
Finished CH04177
Reached maturity at 2018-10-24 with total biomass 17738.27 kg/ha and a yield of 11027.97 kg/ha.
Finished CH04178
Reached maturity at 2018-10-28 with total biomass 18376.75 kg/ha and a yield of

Finished CH04242
Reached maturity at 2018-09-12 with total biomass 19379.12 kg/ha and a yield of 10644.09 kg/ha.
Finished CH04243
Reached maturity at 2018-09-26 with total biomass 19932.02 kg/ha and a yield of 10888.40 kg/ha.
Finished CH04244
Reached maturity at 2018-10-01 with total biomass 19803.38 kg/ha and a yield of 10722.07 kg/ha.
Finished CH04245
Reached maturity at 2018-10-16 with total biomass 20079.11 kg/ha and a yield of 10265.13 kg/ha.
Finished CH04246
Reached maturity at 2018-09-27 with total biomass 19745.10 kg/ha and a yield of 10855.01 kg/ha.
Finished CH04247
Reached maturity at 2018-09-20 with total biomass 19718.51 kg/ha and a yield of 11113.44 kg/ha.
Finished CH04248
Reached maturity at 2018-09-16 with total biomass 19540.87 kg/ha and a yield of 10938.32 kg/ha.
Finished CH04249
Reached maturity at 2018-09-20 with total biomass 19719.37 kg/ha and a yield of 11113.47 kg/ha.
Finished CH04250
Reached maturity at 2018-10-07 with total biomass 20452.82 kg/ha and a yield of